# Graph neural networks



In [ ]:
import os

import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch_geometric.data import Data
from torch_geometric.data.separate import separate
from torch_geometric.datasets import Planetoid, ZINC
from torch_geometric.loader import DataLoader, LinkNeighborLoader
from torch_geometric.utils import scatter
from torch_geometric.nn import GCNConv, GATConv, TransformerConv
from torch_geometric.utils import to_dense_adj

import plotly.express as px

## Data for node prediction

A now classic dataset for GNNs (one whose use is also discouraged within some circles as it is very easy to overfit on), Cora is a nice small dataset to start looking at GNNs. There are many variations of the Cora dataset originally presented in "Automating the Construction of Internet Portals with Machine Learning" by McCallum et al. (https://link.springer.com/article/10.1023/A:1009953814988).

We will use the Cora dataset variant as presented in “FastGCN: Fast Learning with Graph Convolutional Networks via Importance Sampling” (https://arxiv.org/abs/1801.10247). It describes a citation network of 2708 papers and our task is classify each paper into one of 7 different categories.

In [ ]:
# Load dataset:
data_dir = "./cora_data_full"
try: 
    os.mkdir(data_dir)
except FileExistsError:
    print(data_dir, 'exists')
    
dataset = Planetoid(data_dir, 'Cora', split="full")

In [ ]:
# Make sure data is valid
data = dataset[0]
data.validate(raise_on_error=True)

In [ ]:
# Describe data
print('Size:', len(dataset))
print('Classes:', dataset.num_classes)
print('Node features:', dataset.num_node_features)
print('Edge features:', dataset.num_edge_features)

print('Total number of nodes:', data.num_nodes)
print('Total number of edges:', data.num_edges)

In [ ]:
print('Train nodes:', torch.sum(data.train_mask))
print('Val nodes:', torch.sum(data.val_mask))
print('Test nodes:', torch.sum(data.test_mask))

In [ ]:
# train-test split
train_x = data.x[data.train_mask]
train_y = data.y[data.train_mask]

valid_x = data.x[data.val_mask]
valid_y = data.y[data.val_mask]

test_x = data.x[data.test_mask]
test_y = data.y[data.test_mask]

In [ ]:
print(f"Train shape x: {train_x.shape}, y: {train_y.shape}")
print(f"Val shape x: {valid_x.shape}, y: {valid_y.shape}")
print(f"Test shape x: {test_x.shape}, y: {test_y.shape}")

## Baseline: feed-forward neural network

In [ ]:
class FFNN(nn.Module):
    """A simple feed forward neural network with no hidden layers

    Args:
        input_dim (int): Dimensionality of the input feature vectors
        output_dim (int): Dimensionality of the output softmax distribution
    """
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.layer_1 = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        x = self.layer_1(x)
        return F.log_softmax(x, dim=1)

In [ ]:
n_epochs = 100
model = FFNN(input_dim=train_x.shape[-1], output_dim=7)
optimiser = torch.optim.Adam(model.parameters(), lr=0.001)

val_accuracy = {key: 0 for key in range(n_epochs)}

for epoch in range(n_epochs):
    model.train()
    optimiser.zero_grad()
    y_hat = model(train_x)
    loss = F.cross_entropy(y_hat, train_y)
    loss.backward()
    optimiser.step()
    
    model.eval()
    y_hat = model(valid_x)
    y_hat = y_hat.data.max(1)[1]
    num_correct = y_hat.eq(valid_y.data).sum()
    num_total = len(valid_y)
    val_accuracy[epoch] = 100.0 * (num_correct.detach().numpy()/num_total)

In [ ]:
fig = px.scatter(x=range(n_epochs), y=np.array(list(val_accuracy.values())))

fig.update_layout(template="plotly_white", 
                  xaxis_title="epoch",
                  yaxis_title="validation accuracy (in %)")

fig.show()

## Do it yourself graph neural network

The generic equation for convolutional GNN is
$$
\mathbf{h_i} = \phi \big(\mathbf{x_i}, \oplus_{j \in \mathcal{N}_i} c_{i,j} \psi (\mathbf{x_j}) \big).
$$
Rewrite this in terms of adjacency matrix $\mathbf{A}$ and implement it.

In [ ]:
class GCNLayer(nn.Module):
    """GCN layer

    Args:
        input_dim (int): Dimensionality of the input feature vectors
        output_dim (int): Dimensionality of the output softmax distribution
        A (torch.Tensor): 2-D adjacency matrix
    """
    def __init__(self, input_dim, output_dim, A):
        super(GCNLayer, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.A = A

        self.linear = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        
        x = torch.matmul(self.A, x)
        x = self.linear(x)
        
        return x


In [ ]:
class GNN(nn.Module):
    """GNN model using different layers

    Args:
        input_dim (int): Dimensionality of the input feature vectors
        output_dim (int): Dimensionality of the output softmax distribution
        A (torch.Tensor): 2-D adjacency matrix
    """
    def __init__(self, input_dim, output_dim, A):
        super().__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.layer = GCNLayer(input_dim, output_dim, A)

    def forward(self, x):
        x = self.layer(x)
        return F.log_softmax(x, dim=1)

In [ ]:
# adjacency matrix
adj = to_dense_adj(data.edge_index)[0]

In [ ]:
n_epochs = 100
model = GNN(input_dim=train_x.shape[-1], output_dim=7, A=adj)
optimiser = torch.optim.Adam(model.parameters(), lr=0.001)

val_accuracy = {key: 0 for key in range(n_epochs)}

for epoch in range(n_epochs):
    model.train()
    optimiser.zero_grad()
    y_hat = model(data.x)[data.train_mask]
    loss = F.cross_entropy(y_hat, train_y)
    loss.backward()
    optimiser.step()
    
    model.eval()
    y_hat = model(data.x)[data.val_mask]
    y_hat = y_hat.data.max(1)[1]
    num_correct = y_hat.eq(valid_y.data).sum()
    num_total = len(valid_y)
    val_accuracy[epoch] = 100.0 * (num_correct.detach().numpy()/num_total)

In [ ]:
print(np.max(list(val_accuracy.values())))

In [ ]:
fig = px.scatter(x=range(n_epochs), y=np.array(list(val_accuracy.values())))

fig.update_layout(template="plotly_white", 
                  xaxis_title="epoch",
                  yaxis_title="validation accuracy (in %)")

fig.show()

## Do it yourself graph neural network - normalization

The generic equation for convolutional GNN is
$$
\mathbf{h_i} = \phi \big(\mathbf{x_i}, \oplus_{j \in \mathcal{N}_i} c_{i,j} \psi (\mathbf{x_j}) \big).
$$

Specific implementations often differ slightly from this form. For example Kipf and Welling's "Semi-Supervised Classification with Graph Convolutional Networks" (https://arxiv.org/abs/1609.02907) employs a symmetric normalisation for the convolution coefficients with a re-normalisation to tackle exploding parameters. Here

$$
\mathbf{H} = \sigma \big( \mathbf{\tilde{D}}^{-\frac{1}{2}} \mathbf{\tilde{A}} \mathbf{\tilde{D}}^{-\frac{1}{2}} \mathbf{X} \mathbf{W} \big),
$$

where $\mathbf{\tilde{A}} = \mathbf{A} + \mathbf{I}$ and $\mathbf{\tilde{D}}$ is the degree matrix of $\mathbf{\tilde{A}}$.

Is the behaviour of the generic, convolutional GNN layer the same as Kipf and Welling's GCN? Implement this version. 

In [ ]:
class GCNLayer(nn.Module):
    """GCN layer

    Args:
        input_dim (int): Dimensionality of the input feature vectors
        output_dim (int): Dimensionality of the output softmax distribution
        A (torch.Tensor): 2-D adjacency matrix
    """
    def __init__(self, input_dim, output_dim, A):
        super(GCNLayer, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.A = A

        self.adj_norm = adj + torch.eye(A.shape[0])
        self.degree = torch.inverse(torch.sqrt(torch.diag(adj.sum(axis=1))))
        self.adj_norm = torch.matmul(torch.matmul(self.degree, self.adj_norm), self.degree)

        self.linear = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        
        x = torch.matmul(self.adj_norm, x)
        x = self.linear(x)
        
        return x


In [ ]:
class GNN(nn.Module):
    """GNN model using different layers

    Args:
        input_dim (int): Dimensionality of the input feature vectors
        output_dim (int): Dimensionality of the output softmax distribution
        A (torch.Tensor): 2-D adjacency matrix
    """
    def __init__(self, input_dim, output_dim, A):
        super().__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.layer = GCNLayer(input_dim, output_dim, A)

    def forward(self, x):
        x = self.layer(x)
        return F.log_softmax(x, dim=1)

In [ ]:
adj = to_dense_adj(data.edge_index)[0]

In [ ]:
n_epochs = 100
model = GNN(input_dim=train_x.shape[-1], output_dim=7, A=adj)
optimiser = torch.optim.Adam(model.parameters(), lr=0.001)

val_accuracy = {key: 0 for key in range(n_epochs)}

for epoch in range(n_epochs):
    model.train()
    optimiser.zero_grad()
    y_hat = model(data.x)[data.train_mask]
    loss = F.cross_entropy(y_hat, train_y)
    loss.backward()
    optimiser.step()
    
    model.eval()
    y_hat = model(data.x)[data.val_mask]
    y_hat = y_hat.data.max(1)[1]
    num_correct = y_hat.eq(valid_y.data).sum()
    num_total = len(valid_y)
    val_accuracy[epoch] = 100.0 * (num_correct.detach().numpy()/num_total)

In [ ]:
print(np.max(list(val_accuracy.values())))

In [ ]:
fig = px.scatter(x=range(n_epochs), y=np.array(list(val_accuracy.values())))

fig.update_layout(template="plotly_white", 
                  xaxis_title="epoch",
                  yaxis_title="validation accuracy (in %)")

fig.show()

## Existing graph neural networks

In [ ]:
class GNN(nn.Module):
    """GNN model using different layers
    Args:
        input_dim (int): Dimensionality of the input feature vectors
        output_dim (int): Dimensionality of the output softmax distribution
        A (torch.Tensor): 2-D adjacency matrix
    """
    def __init__(self, input_dim, output_dim, layer):
        super().__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.layer = layer(input_dim, output_dim)

    def forward(self, x):
        x = self.layer(x, data.edge_index)
        return F.log_softmax(x, dim=1)

In [ ]:
n_epochs = 150

layers = {'GCNConv': GCNConv, 'GATConv': GATConv, 'TransformerConv': TransformerConv}

model_accuracies = {}
for key, layer in layers.items():
    model = GNN(input_dim=train_x.shape[-1], output_dim=7, layer=layer)
    optimiser = torch.optim.Adam(model.parameters(), lr=0.001)
    val_accuracy = {key: 0 for key in range(n_epochs)}
    
    for epoch in range(n_epochs):
        model.train()
        optimiser.zero_grad()
        y_hat = model(data.x)[data.train_mask]
        loss = F.cross_entropy(y_hat, train_y)
        loss.backward()
        optimiser.step()
        
        model.eval()
        y_hat = model(data.x)[data.val_mask]
        y_hat = y_hat.data.max(1)[1]
        num_correct = y_hat.eq(valid_y.data).sum()
        num_total = len(valid_y)
        val_accuracy[epoch] = 100.0 * (num_correct.detach().numpy()/num_total)
        
    model_accuracies[key] = val_accuracy

In [ ]:
val_acc = pd.DataFrame(np.array([np.array(list(acc.values())) for acc in model_accuracies.values()]).transpose(), 
                       columns=layers.keys())
val_acc.loc[:, 'epoch'] = range(n_epochs)

print('Maximal validation accuracies:\n', val_acc.set_index('epoch').max(axis=0), sep='')

val_acc = val_acc.melt(id_vars='epoch', value_name='validation accuracy', var_name='layer')

In [ ]:
fig = px.scatter(data_frame=val_acc, x='epoch', y='validation accuracy', color='layer')

fig.update_layout(template="plotly_white", 
                  xaxis_title="epoch",
                  yaxis_title="validation accuracy (in %)")

fig.show()

## Data for graph-level prediction

For this, we will use the ZINC dataset for graph-regression. It contains about 12 000 molecular graphs with up to 38 nodes each and the task is to predict for each molecule the solubility (a scalar number).

In [ ]:
# Load dataset:
data_dir = "./zinc_data"
try: 
    os.mkdir(data_dir)
except FileExistsError:
    print(data_dir, 'exists')

train_zinc_dataset = ZINC(root=data_dir, split='train', subset=True)
val_zinc_dataset = ZINC(root=data_dir, split='val', subset=True)
test_zinc_dataset = ZINC(root=data_dir, split='test', subset=True)

In [ ]:
# Data description
print(f"\nTrain examples: {len(train_zinc_dataset)}")
print(f"Val examples: {len(val_zinc_dataset)}")
print(f"Test examples: {len(test_zinc_dataset)}\n")

# Description of first graph
one_graph = train_zinc_dataset[0]
print(f"First graph contains {one_graph.x.shape[0]} nodes, each characterised by {one_graph.x.shape[1]} features")
print(f"Graph labels have shape: {one_graph.y.shape}")

### GNN for graph-level prediction

In [ ]:
def extract(x, idx, slices):
    start, end = int(slices[idx]), int(slices[idx + 1])
    return x.narrow(0, start, end-start).sum(axis=0).reshape(1, -1)

class GNN(nn.Module):
    """GNN model using different layers
    Args:
        input_dim (int): Dimensionality of the input feature vectors
        output_dim (int): Dimensionality of the output distribution
        A (torch.Tensor): 2-D adjacency matrix
    """
    def __init__(self, input_dim, mid_dim, output_dim, layer):
        super().__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.graph_layer = layer(input_dim, mid_dim)
        self.linear = nn.Linear(mid_dim, output_dim)

    def forward(self, x, edge_index, ptr):
        x = self.graph_layer(x, edge_index)
        x = F.relu(x)
        x = torch.concat([extract(x, idx, ptr) for idx in range(len(ptr)-1)], axis=0)
        x = self.linear(x)
        x = F.leaky_relu(x)
        return x.flatten()

In [ ]:
loader = DataLoader(train_zinc_dataset, batch_size=16)
valloader = DataLoader(val_zinc_dataset, batch_size=16)
y_val = np.array([graph.y.detach().numpy() for graph in val_zinc_dataset])

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GNN(input_dim=1, mid_dim=8, output_dim=1, layer=GCNConv).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()

model_rmses = {}
n_epochs = 20
for epoch in range(n_epochs): 
    model.train()
    for batch in loader:
        data = batch.to(device)
        optimizer.zero_grad()
        out = model(data.x.float(), data.edge_index, data.ptr)
        loss = F.mse_loss(out, data.y)
        loss.backward()
        optimizer.step()
    with torch.no_grad():
        outputs = np.hstack([np.array(model(batch.x.float(), batch.edge_index, batch.ptr)) for batch in valloader]).reshape(-1, 1)
        model_rmses[epoch] = mean_squared_error(y_val, outputs, squared=False)

In [ ]:
fig = px.scatter(x=range(n_epochs), y=np.array(list(model_rmses.values())))

fig.update_layout(template="plotly_white", 
                  xaxis_title="epoch",
                  yaxis_title="validation RMSE")

fig.show()